# Transformer (Translation)

In [1]:
import random
import numpy as np
import tensorflow as tf
from konlpy.tag import Okt

from tensorflow.keras import Model
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Lambda, Layer, Embedding, LayerNormalization

In [ ]:
%pwd

In [2]:
import os
os.chdir('/home/jeyong/Disks/hdd1/workspace/Jupyter/deeplearning_part3')

In [3]:
EPOCHS = 200
NUM_WORDS = 2000

## Dot-Scaled Attention

In [4]:
class DotScaledAttention(Layer):
    def __init__(self, d_emb, d_reduced, masked=False):
        super().__init__()
        pass

    def call(self, x, training=None, mask=None): # (q,k,v)
        

        if self.masked: # Referred from https://github.com/LastRemote/Transformer-TF2.0
            length = tf.shape(comp)[-1]
            mask = tf.fill((length, length), -np.inf)
            mask = tf.linalg.band_part(mask, 0, -1) # Get upper triangle
            mask = tf.linalg.set_diag(mask, tf.zeros((length))) # Set diagonal to zeros to avoid operations with infinity
        
        pass

## Multi-Head Attention

In [5]:
class MultiHeadAttention(Layer):
    def __init__(self, num_head, d_emb, d_reduced, masked=False):
        pass

    def call(self, x, training=None, mask=None):
        pass

## Encoder

In [6]:
class Encoder(Layer):
    def __init__(self, num_head, d_reduced):
        super().__init__()
        pass
    
    def build(self, input_shape):
        pass
        super().build(input_shape)

    def call(self, x, training=None, mask=None):
        pass

    def compute_output_shape(self, input_shape):
        return input_shape

## Decoder

In [7]:
class Decoder(Layer):
    def __init__(self, num_head, d_reduced):
        super().__init__()
        pass
    
    def build(self, input_shape):
        pass
        super().build(input_shape)

    def call(self, inputs, training=None, mask=None):
        pass

    def compute_output_shape(self, input_shape):
        return input_shape

## Positional Encoding

In [8]:
class PositionalEncoding(Layer):  # Referred from https://github.com/LastRemote/Transformer-TF2.0
    def __init__(self, max_len, d_emb):
        super().__init__()
        self.sinusoidal_encoding = np.array([self.get_positional_angle(pos, d_emb) for pos in range(max_len)], dtype=np.float32)
        self.sinusoidal_encoding[:, 0::2] = np.sin(self.sinusoidal_encoding[:, 0::2])
        self.sinusoidal_encoding[:, 1::2] = np.cos(self.sinusoidal_encoding[:, 1::2])
        self.sinusoidal_encoding = tf.cast(self.sinusoidal_encoding, dtype=tf.float32)

    def call(self, x, training=None, mask=None):
        return x + self.sinusoidal_encoding[:tf.shape(x)[1]]

    def compute_output_shape(self, input_shape):
        return input_shape

    def get_angle(self, pos, dim, d_emb):
        return pos / np.power(10000, 2 * (dim // 2) / d_emb)

    def get_positional_angle(self, pos, d_emb):
        return [self.get_angle(pos, dim, d_emb) for dim in range(d_emb)]

## Transformer Architecture

In [9]:
class Transformer(Model):
    def __init__(self, src_vocab, dst_vocab, max_len, d_emb, d_reduced, n_enc_layer, n_dec_layer, num_head):
        super().__init__()
        pass

    def call(self, inputs, training=None, mask=None):
        pass

## Dataset 준비

In [10]:
dataset_file = 'chatbot_data.csv' # acquired from 'http://www.aihub.or.kr' and modified
okt = Okt()

with open(dataset_file, 'r') as file:
    lines = file.readlines()
    seq = [' '.join(okt.morphs(line)) for line in lines]

questions = seq[::2]
answers = ['\t ' + lines for lines in seq[1::2]]

num_sample = len(questions)

perm = list(range(num_sample))
random.seed(0)
random.shuffle(perm)

train_q = list()
train_a = list()
test_q = list()
test_a = list()

for idx, qna in enumerate(zip(questions, answers)):
    q, a = qna
    if perm[idx] > num_sample//5:
        train_q.append(q)
        train_a.append(a)
    else:
        test_q.append(q)
        test_a.append(a)

tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=NUM_WORDS,
                                                  filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~')

tokenizer.fit_on_texts(train_q + train_a)

train_q_seq = tokenizer.texts_to_sequences(train_q)
train_a_seq = tokenizer.texts_to_sequences(train_a)

test_q_seq = tokenizer.texts_to_sequences(test_q)
test_a_seq = tokenizer.texts_to_sequences(test_a)

x_train = tf.keras.preprocessing.sequence.pad_sequences(train_q_seq,
                                                        value=0,
                                                        padding='pre',
                                                        maxlen=64)
y_train = tf.keras.preprocessing.sequence.pad_sequences(train_a_seq,
                                                        value=0,
                                                        padding='post',
                                                        maxlen=65)
y_train_shifted = np.concatenate([np.zeros((y_train.shape[0], 1)), y_train[:, 1:]], axis=1)


x_test = tf.keras.preprocessing.sequence.pad_sequences(test_q_seq,
                                                       value=0,
                                                       padding='pre',
                                                       maxlen=64)
y_test = tf.keras.preprocessing.sequence.pad_sequences(test_a_seq,
                                                       value=0,
                                                       padding='post',
                                                       maxlen=65)

## Train using keras

In [ ]:
transformer = Transformer(NUM_WORDS, NUM_WORDS, 128, 16, 16, 2, 2, 4) # Instantiating a new transformer model

transformer.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
transformer.fit([x_train, y_train_shifted], y_train, batch_size=5, epochs=EPOCHS)

Train on 399 samples
Epoch 1/200
399/399 [==============================] - 9s 22ms/sample - loss: 5.9527 - accuracy: 0.8479
Epoch 2/200
399/399 [==============================] - 1s 4ms/sample - loss: 2.7419 - accuracy: 0.8900
Epoch 3/200
399/399 [==============================] - 2s 4ms/sample - loss: 1.0273 - accuracy: 0.8900
Epoch 4/200
399/399 [==============================] - 1s 4ms/sample - loss: 0.7265 - accuracy: 0.8900
Epoch 5/200
399/399 [==============================] - 1s 3ms/sample - loss: 0.6082 - accuracy: 0.8991
Epoch 6/200
399/399 [==============================] - 1s 3ms/sample - loss: 0.5347 - accuracy: 0.9207
Epoch 7/200
399/399 [==============================] - 1s 3ms/sample - loss: 0.4885 - accuracy: 0.9238
Epoch 8/200
399/399 [==============================] - 1s 3ms/sample - loss: 0.4585 - accuracy: 0.9265
Epoch 9/200
399/399 [==============================] - 1s 3ms/sample - loss: 0.4344 - accuracy: 0.9265
Epoch 10/200
399/399 [=============================